In [3]:
import sys

sys.path.append("../")

from utils import open_pickle
from utils import check_solution
from cube3_game import Cube3Game

import numpy as np
import torch

from g_datasets import *

In [2]:
optimum = open_pickle("../assets/data/deepcubea/data_0.pkl")
deepcube_res = open_pickle("../assets/data/deepcubea/results.pkl")
report = open_pickle("../assets/reports/Cube3ResnetModel_value_policy_3_8B_14M_search_value_full.pkl")
game = Cube3Game("../assets/envs/qtm_cube3.pickle")
generators = torch.tensor(game.actions, dtype=torch.int64)

In [ ]:
state_size = game.actions.shape[1]
hash_vec = torch.randint(0, 1_000_000_000_000, (state_size,))  

In [ ]:
scrambles, actions, lengths = get_torch_scrambles_3(
    N=1000,
    n=30,
    generators=generators,
    hash_vec=hash_vec,
    device="cpu"
)
